# Question 2
Part 1
Read all the json files in the folder called Data.
 There are three categories of json files in this folder. They are identified by the key
called “term” in each of the json file.
 Create a folder structure to read all these json files and store them into these separate
folders. You are expected to create a hierarchy of folder structure.
 Example:
 You can place all restaurants json files in a particular country (say Australia) in the
same folder. How you group the json files and create a folder structure is your choice.
Your task is to identify criteria by which you can group all these json files and store
them.
 (You could use these keys to create hierarchy and store json files: Country, city,
categories)

In [2]:
import shutil
import os
import json
import glob
import csv

In [4]:
root = os.getcwd()
rootPathjson = os.path.join('.', 'Data', '')
filenames = [path_json for path_json in os.listdir(rootPathjson) if path_json.endswith('.json')]

.\Data\


In [102]:
for f in filenames:
    old_file_path = os.path.join(rootPathjson, f)
    with open(old_file_path) as json_file:
        # Load the content of the file       
        json_content = json.load(json_file)
        # Select country from json content
        country = json_content['location']['country']
        # Select city from json content       
        city = json_content['location']['city']
        # Select termfrom json content        
        term = json_content['term']
          
        # Create a new directory path as mentioned
        new_dir_path = os.path.join('.', 'Data Processed', country, city, term, '')
        
        os.makedirs(new_dir_path, exist_ok=True)
        
        new_file_path = os.path.join(new_dir_path, f)
                
        # Preserve the contents of the Data folder by copying rather than moving
        shutil.copy2(old_file_path, new_file_path)

# Part 2
Part 2
Read all the json files in the folder called Data.
 Read only the json files which contain the key called “restaurants”
 Each (or most of the json files) contain a key called “open” which contains the details
of the operation (timings) of the restaurants. For each json file, read the timings of the
restaurants.
 Data of the operation timings of the restaurants is present for each day of the week. I
want you to extract each of this data and write it in an excel sheet.

In [121]:
information_file = []
for f in filenames:
    file_path = os.path.join(rootPathjson, f)
    with open(file_path) as json_file:
        # Load the content of the file.        
        json_content = json.load(json_file)
        
        country = json_content['location']['country']
        city = json_content['location']['city']
        term = json_content['term']
        if term == 'restaurants':
            information_file.append(file_path)

In [122]:
# json_content is the file content, created using json.load(file)
# row_list is the list of rows to which a row will be added; later used to create a dataframe
# exception_list is the list of files that couldn't be processed
def process_restaurant_file_json(json_content, row_list, exception_list):
    name = json_content['name']
    city = json_content['location']['city']
    country = json_content['location']['country']        

    if 'hours' in json_content:
        for hours in json_content['hours']:
            if hours['hours_type'] == 'REGULAR':
                for h in hours['open']:
                    day = str(h['day'])
                    start = str(h['start'])
                    end = str(h['end'])
          
                    start_hour = start[0:2]
                    start_minutes = start[2:4]

                    end_hour = end[0:2]
                    end_minutes = end[2:4]

                    if (len(start_hour) != 2) or (len(start_minutes) != 2) or (len(end_hour) != 2) or (len(end_minutes) != 2):
                        print(start_hour + ' ' + start_minutes + ' ' + end_hour + ' ' + end_minutes)
                    row = {'Name of restaurant':name, 'City':city, 'Country Code':country, 
                           'Day of week':day, 'Start time hour':start_hour, 'Start time minute':start_minutes, 
                           'End time hour':end_hour, 'End time minute':end_minutes}
                    row_list.append(row)
            else:
                print(name + ' has hours_type of ' + hours['hours_type'])
    else:
        # Save the exceptions for separate examination.
        # The files in the exception list have no 'hours'
        exception_list.append(rf)
        
    return row_list, exception_list

In [123]:
exceptions = []
row_list = []

for restohrs in information_file:
    with open(restohrs) as json_file:
        # Load the content of the file.        
        json_content = json.load(json_file)
        
        # Process the JSON content.
        row_list, exceptions = process_restaurant_file_json(json_content, row_list, exceptions)

# Create the dataframe from the list of rows.
df = pd.DataFrame(row_list, columns=['Name of restaurant', 'City', 'Country Code', 'Day of week', 
                           'Start time hour', 'Start time minute', 'End time hour', 'End time minute'])

# Write the CSV file.
df.to_csv(os.path.join('.', 'restoHrs.csv'), index=False)